# EFX algorithms for homogeneous items

In [1]:
import numpy as np

In [2]:
def check_all_efx(assignments, vals):
    for j in range(len(assignments)):
        if not check_efx(assignments, vals, j):
            return False
    return True

def check_efx(assignments, vals, j):
    my_value = vals[j].dot(assignments[j])
    for i in range(len(assignments)):
        ass = assignments[i]
        ef_condition = vals[j].dot(ass) <= my_value
        if ef_condition:
            continue

        for l in range(len(ass)):
            if ass[l] > 0:
                new_vec = list(ass[:l]) + [ass[l] - 1] + list(ass[l+1:])
                condition = vals[j].dot(new_vec) <= my_value
                if not condition:
                    return False
    return True

def get_all_envy(assignments, vals):
    return np.array([get_envy(assignments, vals, j) for j in range(len(assignments))])

def get_envy(assignments, vals, j):
    my_value = vals[j].dot(assignments[j])
    envied = []
    for i in range(len(assignments)):
        ass = assignments[i]
        ef_delta = vals[j].dot(ass) - my_value
        envied.append(ef_delta)
    return envied

def get_envied_by(assignments, vals, i):
    envied_by = []
    ass = assignments[i]
    for j in range(len(assignments)):
        ef_delta = vals[j].dot(ass) - vals[j].dot(assignments[j])
        envied_by.append(ef_delta)
    return envied_by

def get_2_cycle(assignments, vals):
    arr = get_all_envy(assignments, vals) > 0
    for i in range(len(arr)):
        for j in range(len(arr)):
            if arr[i][j] and arr[j][i]:
                return i,j
    return False

# Algorithm for EFX at t=2

In [3]:
def scale_inputs(valuations, items):
    valuations = np.array(valuations)

    top_prefs = np.argmax(valuations, axis=1)
    num_ppl = [sum(top_prefs == i) for i in range(len(items))]

    if items[0]/num_ppl[0] < items[1]/num_ppl[1]:
        vals = valuations[:,:]
    else:
        items = items[::-1]
        num_ppl = num_ppl[::-1]
        vals = valuations[:,::-1]
        
    # scale everything
    vals[:,0] /= vals[:,1]
    vals[:,1] = 1
    
    # change sort order
    order = np.argsort(vals[:,0])
    vals = vals[order, :][::-1, :]

    return vals, items

In [4]:
def efx(vals, items, print_statements=False):
    pr = print if print_statements else lambda x: x
    n_players = len(vals)
    assignments = np.array([(0,0)]*n_players)
    items_left = np.copy(items)
    top_prefs = np.argmax(vals, axis=1)
    num_ppl = [sum(top_prefs == i) for i in range(len(items))]
    assert check_all_efx(assignments, vals), "NOT WORKING IN BEGINNING"
    
    # assign one item each
    while items_left[0] >= num_ppl[0]:
        pr(items_left)
        for idx in range(len(vals)):
            if vals[idx,0] > 1:
                # a-type person
                items_left[0] -= 1
                assignments[idx][0] += 1
            else:
                items_left[1] -= 1
                assignments[idx][1] += 1
            assert check_all_efx(assignments, vals), "NOT WORKING IN INITIAL ROUND ROBIN"
    pr(items_left)
    
    # do leftovers:
    leftover_ct = items_left[0]
    for idx in range(leftover_ct):
        assert vals[idx,0] > 1
        items_left[0] -= 1
        assignments[idx][0] += 1
        assert check_all_efx(assignments, vals), "NOT WORKING ASSIGNING LEFTOVERS"
    pr(items_left)
    
    # round robin skipping leftovers
    idx = leftover_ct
    num_rounds = 0
    while leftover_ct != 0 and items_left[1] > 0 and num_rounds < np.ceil(vals[leftover_ct-1,:][0]):
        items_left[1]-=1
        assignments[idx][1] += 1
        assert check_all_efx(assignments, vals), "NOT WORKING ROUND ROBIN AFTER LEFTOVERS " + str(idx) + " " + str(num_rounds)
        idx += 1
        if idx == n_players:
            pr(items_left)
            idx = leftover_ct
            num_rounds += 1
            
    # round robin for all
    idx = 0
    while items_left[1] > 0:
        items_left[1] -=1
        assignments[idx][1] += 1
        assert check_all_efx(assignments, vals), "NOT WORKING FINAL ROUND ROBIN"
        idx += 1
        if idx == n_players:
            pr(items_left)
            idx = 0
    
    pr(items_left)
    assert check_all_efx(assignments, vals), "EFX SHOULD HOLD"
    assert sum(items_left) == 0, "SHOULD USE ALL ITEMS"
    assert np.all(np.sum(assignments, axis=0) == items), "ALL ITEMS ALLOCATED"
    
    return assignments

### Testing it out

In [5]:
# t1, t2 valuations
orig_valuations = [
    (3,2),
    (2,1),
    (0.5,1),
    (4,1),
]

orig_items = [20, 20]

In [6]:
vals, items = scale_inputs(orig_valuations, orig_items)
print(vals, items)
efx(vals, items, print_statements=True)

[[4.  1. ]
 [2.  1. ]
 [1.5 1. ]
 [0.5 1. ]] [20, 20]
[20 20]
[17 19]
[14 18]
[11 17]
[ 8 16]
[ 5 15]
[ 2 14]
[ 0 14]
[ 0 12]
[ 0 10]
[0 6]
[0 2]
[0 0]


array([[ 7,  3],
       [ 7,  3],
       [ 6,  4],
       [ 0, 10]])

In [7]:
%%time
max_items = 40
for i in range(max_items):
    for j in range(max_items):
        vals, items = scale_inputs(orig_valuations, [i,j])
        efx(vals, items)

CPU times: user 2.59 s, sys: 6.87 ms, total: 2.6 s
Wall time: 2.6 s


Try random valuations too

In [8]:
%%time
trials = 2
n_players = 5
max_items = 40
for _ in range(trials):
    vals = [(np.random.random(), np.random.random()) for _ in range(n_players)]
    for i in range(max_items):
        for j in range(max_items):
            vals, items = scale_inputs(orig_valuations, [i,j])
            efx(vals, items)

CPU times: user 4.61 s, sys: 835 µs, total: 4.61 s
Wall time: 4.61 s


# Now: EFX t=3 (still a draft)

In [9]:
def scale_inputs_t3(valuations, items):
    valuations = np.array(valuations)

    top_prefs = np.argmax(valuations, axis=1)
    num_ppl = [sum(top_prefs == i) for i in range(len(items))]

    # for now, sort by # items / # favorite players to get type a
    sort_order = np.argsort([items[i]/num_ppl[i] for i in range(len(items))])
    items = np.array(items)[sort_order]
    num_ppl = np.array(num_ppl)[sort_order]
    vals = valuations[:, sort_order]

    # change sort order to be by strength of a-type over next-favorite type
    order = np.argsort(vals[:,0]/np.max(vals[:, 1:], axis=1))
    vals = vals[order, :][::-1, :]

    return vals, items

In [488]:
def efx_t3(vals, items, print_statements=True, print_counts=True, print_output=True):
    pr = print if print_statements else lambda *x: x
    pr2 = print if print_counts else lambda *x: x
    pr3 = print if print_output else lambda *x: x
    n_players = len(vals)
    assignments = np.array([[0]*len(items)]*n_players)
    items_left = np.copy(items)
    top_prefs = np.argmax(vals, axis=1)
    pr("type a:", np.where(top_prefs == 0)[0])
    pr("type b:", np.where(top_prefs == 1)[0])
    pr("type c:", np.where(top_prefs == 2)[0])

    num_ppl = [sum(top_prefs == i) for i in range(len(items))]
    assert check_all_efx(assignments, vals), "NOT WORKING IN BEGINNING"

    pr("everyone gets an item")
    # assign one item each
    while items_left[0] >= num_ppl[0]:
        pr2(items_left)
        for idx in range(len(vals)):
            pref = np.argmax(vals[idx])
            items_left[pref] -= 1
            assignments[idx][pref] += 1
            assert check_all_efx(assignments, vals), "NOT WORKING IN INITIAL ROUND ROBIN"
    pr2(items_left)

    # assign leftovers
    leftover_ct = items_left[0]
    pr("num a leftovers:", leftover_ct)
    for idx in range(leftover_ct):
        assert np.argmax(vals[idx]) == 0
        items_left[0] -= 1
        assignments[idx][0] += 1
        assert check_all_efx(assignments, vals), "NOT WORKING ASSIGNING LEFTOVERS"
    pr2(items_left)

    pr("round robin without A+")
    # do round robin until full
    idx = leftover_ct
    num_rounds = 0
    q_j = vals[leftover_ct-1][0] / np.max(vals[leftover_ct-1][1:])
    while leftover_ct != 0 and num_rounds < np.ceil(q_j):
        if items_left[1] == 0 or items_left[2] == 0:
            pr3("REMOVE THIS ASSUMPTION LATER (section 3.3)")
            return

        pref = np.argmax(vals[idx][1:]) + 1
        assert pref != 0
        items_left[pref] -=1
        assignments[idx][pref] += 1
        assert check_all_efx(assignments, vals), "NOT WORKING ROUND ROBIN AFTER LEFTOVERS " + str(idx) + " " + str(num_rounds)
        idx += 1
        if idx == n_players:
            pr2(items_left)
            idx = leftover_ct
            num_rounds += 1


    pr("round robin with everyone")
    top_prefs_no_a = np.argmax(vals[:,1:], axis=1) + 1
    num_ppl_no_a = [sum(top_prefs_no_a == i) for i in range(len(items))]
    # assign one item each until one item is nearly gone
    while items_left[1] >= num_ppl_no_a[1] and items_left[2] >= num_ppl_no_a[2]:
        pr2(items_left)
        for idx in range(len(vals)):
            pref = np.argmax(vals[idx, 1:])+1
            items_left[pref] -= 1
            assignments[idx][pref] += 1
            assert check_all_efx(assignments, vals), "NOT WORKING IN NO-A ROUND ROBIN"
    pr2(items_left)

    type_a = 0
    type_b = 1 if (items_left[1] < num_ppl_no_a[1]) else 2
    type_c = 3 - type_b
    pr("type b:", type_b)

    pr("num b leftovers:", items_left[type_b])
    a_plus_c_ppl = np.where(vals[:leftover_ct,type_b]<vals[:leftover_ct,type_c])[0]
    a_plus_b_ppl = np.where(vals[:leftover_ct,type_b]>=vals[:leftover_ct,type_c])[0]
    pr("a_plus_b_ppl:", a_plus_b_ppl)
    pr("a_plus_c_ppl:", a_plus_c_ppl)
    if items_left[type_b] == 0: 
        pr("no b leftovers, wow!")
        pr3("REMOVE THIS ASSUMPTION LATER (no b-leftovers) (Section 3.2)")
        return
    else:
        # first give out to the A+B and A+C ppl
        pr("give b leftovers to A+ ppl")
        for idx in range(leftover_ct):
            assert np.argmax(vals[idx]) == 0
            pref = np.argmax(vals[idx, 1:])+1
            if items_left[pref] == 0:
                pr3("REMOVE THIS ASSUMPTION LATER (Section 3.2)")
                return
            items_left[pref] -= 1
            assignments[idx][pref] += 1
            assert check_all_efx(assignments, vals), "NOT WORKING ASSIGNING B LEFTOVERS TO PLUS PPL"
        pr2(items_left)

    # sort by R_j
    pr("R_j values:", np.sort(vals[:,type_b]/vals[:, type_c])[::-1])
    pr("All sorted by R_j:", np.argsort(vals[:,type_b]/vals[:, type_c])[::-1])
    new_sort_order = np.argsort(vals[leftover_ct:,type_b]/vals[leftover_ct:, type_c])[::-1] + leftover_ct
    pr("Non A+ Sorted by R_j:", new_sort_order)
    # now give out remaining type-b items.
    # assign leftovers
    extra_b_leftover_ct = items_left[type_b]
    extra_b_leftover_ppl = new_sort_order[:extra_b_leftover_ct]

    a_minus_b_plus_ppl = extra_b_leftover_ppl[np.where(vals[extra_b_leftover_ppl,type_b]<vals[extra_b_leftover_ppl,type_a])[0]]
    b_plus_ppl = extra_b_leftover_ppl[np.where(vals[extra_b_leftover_ppl,type_b]>=vals[extra_b_leftover_ppl,type_a])[0]]
    pr("a_minus_b_plus_ppl:", a_minus_b_plus_ppl)
    pr("b_plus_ppl:", b_plus_ppl)

    pr("giving b leftovers")
    for idx in extra_b_leftover_ppl:
        pref = np.argmax(vals[idx, 1:])+1
        assert pref == type_b
        items_left[pref] -= 1
        assignments[idx][pref] += 1
        assert check_all_efx(assignments, vals), "NOT WORKING ASSIGNING B LEFTOVERS TO MINUS PPL"
    pr2(items_left)

    # now give c's to to rest of players
    other_players = new_sort_order[extra_b_leftover_ct:]
    pr('other players:', other_players)
    pr("giving cs round robin without B+, AB+")
    for idx in other_players:
        if items_left[type_c] == 0:
            break
        items_left[type_c] -= 1
        assignments[idx][type_c] += 1
#         print(assignments)
        assert check_all_efx(assignments, vals), "NOT WORKING ASSIGNING C to rest of players (first time)"
    pr2(items_left)


    # now give c's to rest of players, and A+C ?
    # should probably loop this until something bad happens
    go_again = True
    do_step_9 = False
    pr("go again loop")
    
    redo_step_8_9 = True
    while redo_step_8_9:
        redo_step_8_9 = False
        while go_again:
            # any A^-B^+,B^+ people envying non b leftover ppl?
            for idx in extra_b_leftover_ppl:
                if np.any(np.array(get_envy(assignments, vals, idx))[other_players] > 0):
                    # is there also reverse envy? THERE REALLY SHOULDNT BE
                    pr("A-B+,B+ envies non-b-leftover ppl:", idx)
                    reverse_envy = False
                    for jdx in other_players:
                        if np.any(np.array(get_envy(assignments, vals, idx))[extra_b_leftover_ppl] > 0):
                            reverse_envy = True
                            break

                    if not reverse_envy:
                        pr("should do step 9")
                        do_step_9 = True
                        go_again = False
                    else:
                        # do cycle
                        assert False, "THERE SHOULDNT BE REVERSE ENVY HERE"

            # any A+B people envying non b leftover ppl?
            did_2cycle = False
            for idx in a_plus_b_ppl:
                if np.any(np.array(get_envy(assignments, vals, idx))[other_players] > 0):
                    pr("A+B envies non-b-leftover ppl:", idx)

                    # is there also reverse envy?
                    reverse_envy = False
                    for jdx in other_players:
                        if np.any(np.array(get_envy(assignments, vals, jdx))[a_plus_b_ppl] > 0):
                            reverse_envy = True
                            break

                    if not reverse_envy:
                        pr("no reverse envy, go to step 10")
                        go_again = False
                    else:
                        # do cycle (just doing the first cycle I ind...)
                        pr("2 CYCLE IN GOAGAIN LOOP")
                        assignments[[idx,jdx]] = assignments[[jdx,idx]]
                        did_2cycle = True
            
            # any other ppl have envy?
            if go_again and not did_2cycle:
                no_envy = True
                for idx in other_players:
                    if np.any(np.array(get_envy(assignments, vals, idx)) > 0):
                        no_envy = False
                if no_envy:
#                     print(get_all_envy(assignments, vals))
                    pr("NO ENVY EXCEPT MAYBE A+B -> A+C or A-B+,B+")
                    go_again = False
                    do_step_9 = True

            
            if items_left[type_c] == 0:
                go_again = False
                do_step_9 = False
                pr("DONE")

            if not go_again:
                break

            if not did_2cycle:
                for idx in a_plus_c_ppl:
                    if items_left[type_c] == 0:
                        break
                    assert np.argmax(vals[idx]) == 0
                    pref = np.argmax(vals[idx, 1:])+1
                    assert pref == type_c
                    items_left[pref] -= 1
                    assignments[idx][pref] += 1
    #                 print(get_all_envy(assignments, vals) > 0)
                    assert check_all_efx(assignments, vals), "NOT WORKING ASSIGNING TO A PLUS C PPL (in go again loop)"
                pr2(items_left)

                # now give c's to to rest of players
                for idx in new_sort_order[extra_b_leftover_ct:]:
                    if items_left[type_c] == 0:
                        break
                    items_left[type_c] -= 1
                    assignments[idx][type_c] += 1
                    assert check_all_efx(assignments, vals), "NOT WORKING ASSIGNING C to rest of ppl (in go again loop): " + str(idx)
                pr2(items_left)





        while do_step_9:
            pr("Step 9")
            # does any player in A+B envy a player in A-B+,B+?
            keep_doing_step_9 = False

            skip_step_9 = True
            for idx in extra_b_leftover_ppl:
                if np.any(np.array(get_envy(assignments, vals, idx))[other_players] > 0):
                    # is there also reverse envy? THERE REALLY SHOULDNT BE
#                     pr("A-B+,B+ envies non-b-leftover ppl:", idx)
                    skip_step_9 = False
            if skip_step_9:
                pr("SKIPPING STEP 9")
                break
            
            do_step_9_check = True
            while do_step_9_check:
                pr("Step 9 check")
                do_step_9_check = False
                for idx in a_plus_b_ppl:
                    if np.any(np.array(get_envy(assignments, vals, idx))[extra_b_leftover_ppl] > 0):
                        keep_doing_step_9 = True
                        pr("A+B envies ppl in A-B+, B+:", idx)

                        if np.any(np.array(get_envied_by(assignments, vals, idx))[other_players] > 0):
                            #print(get_all_envy(assignments, vals) > 0)
                            # TODO do this smartly
                            did_cycle = False
                            bplus_idxs = extra_b_leftover_ppl[np.where(np.array(get_envy(assignments, vals, idx))[extra_b_leftover_ppl] > 0)[0]]
                            op_idxs = other_players[np.where(np.array(get_envied_by(assignments, vals, idx))[other_players] > 0)[0]]
                            for b in bplus_idxs:
                                i_envy = np.where(np.array(get_envy(assignments, vals, b)) > 0)[0]
                                for i in i_envy:
                                    if i in op_idxs:
                                        print("CYCLE: " + " ".join(map(str, [i, b, idx])))
                                        assignments[[i,b,idx]] = assignments[[idx,i,b]]
                                        did_cycle = True
                                        break
                                if did_cycle:
                                    break
                            if did_cycle:
                                do_step_9_check = True
                                redo_step_8_9 = True
                            else:
                                print(get_all_envy(assignments, vals) )
                                assert False, "DIDNT FIND THE step 9 CYCLE?"

                for idx in a_plus_b_ppl:
                    # Check for cycles in reverse order to help with assignment
                      if np.any(np.array(get_envy(assignments, vals, idx))[extra_b_leftover_ppl] > 0):
                        keep_doing_step_9 = True
                        ## NEW: CHECK FOR A 2-CYCLE
                        cycle = get_2_cycle(assignments, vals)
                        if cycle:
#                             print(get_all_envy(assignments, vals) > 0)
                            assert False, "2CYCLE IN STEP 9 - WHAT TO DO?" + str(cycle)
#                             print("2CYCLE in STEP 9", cycle)
#                             cycle = np.array(cycle)
#                             assert cycle[0] == idx or cycle[1] == idx, "WHO IS THIS CYCLE??"
#                             assignments[cycle] = assignments[cycle[::-1]]
#                             do_step_9_check = True
                    
                    
                    
                    
                    
                    
                    
            if redo_step_8_9:
                do_step_9 = False
            elif keep_doing_step_9:
                # give one item to each A+B person
                pr("A+B ppl get a c item")
                for idx in a_plus_b_ppl:
                    if items_left[type_c] == 0:
                        keep_doing_step_9 = False
                        break
                    items_left[type_c] -= 1
                    assignments[idx][type_c] += 1
                    pr2(items_left)
                    assert check_all_efx(assignments, vals), "NOT WORKING ASSIGNING C in step 9"
                do_step_9 = keep_doing_step_9
            else:
                do_step_9 = False
            
    pr("Step 10")
    while items_left[type_c] > 0:
#         print(get_all_envy(assignments, vals))
        # assign to type c through the list, as long as they are not currently envied
        everyone_envied = True
        for j in range(n_players):
            if items_left[type_c] == 0:
                break
            if np.all(np.array(get_envied_by(assignments, vals, j)) <= 0):
                everyone_envied =  False
                items_left[type_c] -=1
                assignments[j][type_c] += 1
                assert check_all_efx(assignments, vals), "NOT WORKING ASSIGNING C in finish line " + str(j)

        if everyone_envied:
            assert False, "STEP 10 CYCLE, what to do???"
#             print("STEP 10 CYCLE")
#             do_again = get_2_cycle(assignments, vals)
#             assert do_again, "SHOULD BE A 2 CYCLE"
#             while do_again:
#                 i,j = get_2_cycle(assignments, vals)
#                 print(i,j)
#                 assignments[[i,j]] = assignments[[j,i]] 
#                 do_again =  get_2_cycle(assignments, vals)
        pr2(items_left)

    assert check_all_efx(assignments, vals), "EFX SHOULD HOLD"
    assert sum(items_left) == 0, "SHOULD USE ALL ITEMS"
    assert np.all(np.sum(assignments, axis=0) == items), "ALL ITEMS ALLOCATED"
    pr3("looks good")
    return assignments

### Testing it out

In [543]:
t = 0
while True:
    t += 1
    if t % 50 == 0:
        print(t)
    n_players = 8
    t3_valuations = [(np.random.random(),np.random.random(),np.random.random()) for i in range(n_players)]
    t3_items = [int(np.random.random()*100) for _ in range(3)]
    vals, items = scale_inputs_t3(t3_valuations, t3_items)
#     print(vals, items)
    assignments = efx_t3(vals, items, print_statements=False, print_counts=False, print_output=False)

<ipython-input-9-9df317c90dd4>:8: RuntimeWarning: divide by zero encountered in long_scalars
  sort_order = np.argsort([items[i]/num_ppl[i] for i in range(len(items))])


50
100
150
200
250


AssertionError: 2CYCLE IN STEP 9 - WHAT TO DO?(2, 3)

## Known Issues

* section 3.2
* section 3.3
* step 10 cycle, or c wrong in the finish line? something is fishy about step 10 -- the A+B --> A-B+,B+ ... step seems wrong, and the envy tolerance is above C still
* step 9 issues: didn't find the step 9 cycle?, step 9 assigning C? (I feel like step 9 doesnt work.. but it doesn't happen that often anymore)
* something wrong giving A+C (I think this is because A+B envies A+C ... what do we do there?)